Currently a work in progress notebook, in future will be full the main notebook.

In [123]:
import wikipedia as wp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part 1:

Retreaving information from Wikipedia on the average temperature in Italy. We will use that to plot it against the number of new cases to visualy demonstrate a correlation between the number of new cases and temperature. 

In [162]:
# Wikipedia scrapper

wiki_page = 'Climate_of_Italy'
html = wp.page(wiki_page).html().replace(u'\u2212', '-')

def dataframe_cleaning(table_number: int):
    global html
    df = pd.read_html(html, encoding='utf-8')[table_number]
    df.drop(np.arange(5, len(df.index)), inplace=True)
    df.columns = df.columns.droplevel()
    df.drop('Year', axis=1, inplace=True)
    
    find = '\((.*?)\)'
    for i, column in enumerate(df.columns):
        if i>0:
            df[column] = (df[column]
                          .str.findall(find)
                          .map(lambda x: (float(x[0])-32)* (5/9)))
    return df

# Setting up all the DataFrames that will be used to plot Italy's average temrpeature with high and low variation. 

potenza_df = dataframe_cleaning(3)
milan_df = dataframe_cleaning(4)
florence_df = dataframe_cleaning(6)

# Constructing aggregated DataFrame for all temperatures in Italy:

concat_df = pd.concat((potenza_df, milan_df, florence_df))

italy_df = pd.DataFrame()
for i, index in enumerate(list(set(concat_df['Month']))):
    if i == 0:
        temp_df = concat_df[concat_df['Month'] == index]
        temp_df = temp_df.groupby('Month').agg(np.max)
    if i in range(1, 4):
        temp_df = concat_df[concat_df['Month'] == index]
        temp_df = temp_df.groupby('Month').agg(np.mean)
    if i == 4:
        temp_df = concat_df[concat_df['Month'] == index]
        temp_df = temp_df.groupby('Month').agg(np.min)
    italy_df = italy_df.append(temp_df)

italy_df = italy_df.apply(lambda x: np.round(x, 2))
italy_df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Month,,,,,,,,,,,,
Daily mean °C (°F),5.72,7.50,10.28,13.00,17.72,21.39,24.61,24.61,20.50,15.50,9.89,6.78
Average low °C (°F),0.57,1.30,3.72,6.43,10.81,14.37,17.04,17.02,13.70,9.33,4.52,1.67
Record low °C (°F),-15.94,-11.83,-7.72,-2.78,1.11,5.07,8.87,8.13,2.61,-1.63,-6.48,-10.00
Average high °C (°F),7.63,9.57,13.24,16.22,21.37,25.41,28.78,28.59,24.28,18.39,12.15,8.46
Record high °C (°F),20.00,21.78,23.61,25.61,29.78,33.00,36.78,36.78,33.00,28.22,21.78,20.00
